In [ ]:
!pip install torch transformers

In [ ]:
pip install torch -U

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.4/906.4 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 81.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 69.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 31.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 37.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 66.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
### IMPORT PACKAGES

import pandas as pd
import re
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from transformers import BertTokenizer, BertForSequenceClassification
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, classification_report, ConfusionMatrixDisplay
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

In [ ]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt_tab')
nltk.download('punkt')
#word_vectors = KeyedVectors.load_word2vec_format('path/to/GoogleNews-vectors-negative300.bin', binary=True)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
### IMPORT DATASET
# from google.colab import drive
# drive.mount('/content/drive')

# df = pd.read_csv('/content/drive/My Drive/AML Project 2/IMDB Dataset.csv')

In [ ]:
### CLEANING AND PREPROCESSING- ADD MORE IF NECESSARY
def text_preprocessing(text):
  text = text.lower()

  text = re.sub(r'[,.!?:()"]', '', text)
  text = re.sub(r'<.*?>', ' ', text)
  text = re.sub(r'http\S+', ' ', text)
  text = re.sub(r'[^a-zA-Z0-9]', ' ', text)
  text = re.sub(r'\s+', ' ', text)

  text = text.strip()

  stop_words = set(stopwords.words('english'))
  text = ' '.join(word for word in text.split() if word not in stop_words)

  tokens = nltk.word_tokenize(text)
  return ' '.join(tokens)

In [ ]:
### PREPROCESSING AND SPLITTING
# preprocess_text = [text_preprocessing(text) for text in X]
# X_dev, X_test, y_dev, y_test = train_test_split(preprocess_text, y, test_size=0.2, random_state=42)

In [ ]:
# Bag of words embedding
def get_bag_of_words_embedding(X_dev, X_test):
    word_embed_bow = CountVectorizer()
    X_dev_bow = word_embed_bow.fit_transform(X_dev)
    X_test_bow = word_embed_bow.transform(X_test)
    return X_dev_bow, X_test_bow

In [ ]:
### WORD EMBEDDINGS - TDIF
def get_tfidf_embedding(X_dev, X_test):
    word_embed_tfidf = TfidfVectorizer()
    X_dev_tfidf = word_embed_tfidf.fit_transform(X_dev)
    X_test_tfidf = word_embed_tfidf.transform(X_test)
    return X_dev_tfidf, X_test_tfidf

In [ ]:
# GloVe embedding
def glove_sentence_embedding(sentence, max_length, embedding_dim, embeddings):
    words = sentence.split()
    num_words = min(len(words), max_length)
    embedding_sentence = np.zeros((max_length, embedding_dim))

    for i in range(num_words):
        word = words[i]
        if word in embeddings.stoi:
            embedding_sentence[i] = embeddings.vectors[embeddings.stoi[word]]

    return embedding_sentence.flatten()

def get_glove_embedding(imdb_df):
    embeddings = GloVe(name='6B', dim=100)

    # Set the maximum sentence length and embedding dimension
    max_length = 100
    embedding_dim = 100

    imdb_df['encode_glove'] = imdb_df['review'].apply(
        lambda sentence: glove_sentence_embedding(sentence, max_length, embedding_dim, embeddings))
    X_glove = np.vstack(imdb_df['encode_glove'])
    return X_glove

In [ ]:
### WORD EMBEDDINGS - BERT
# Bert embedding
def get_bert_embedding(imdb_df):
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    model = BertModel.from_pretrained('bert-base-uncased')

    imdb_df['cls_bert'] = imdb_df['review'].apply(
        lambda sentence: get_cls_sentence(sentence, tokenizer, model))
    X_cls_bert = np.vstack(imdb_df['cls_bert'])
    return X_cls_bert


def get_cls_sentence(sentence, tokenizer, model):
    # Tokenize input sentence and convert to tensor
    input_ids = torch.tensor([tokenizer.encode(sentence, add_special_tokens=True, max_length=512)])

    # Pass input through BERT model and extract embeddings for [CLS] token
    with torch.no_grad():
        outputs = model(input_ids)
        cls_embedding = outputs[0][:, 0, :]

    return cls_embedding.flatten()

In [ ]:
# Read in embedding file, 'review' column already preprocessed - bert column: 'cls_bert', glove column: 'encode_glove'
# imdb_df = pd.read_csv('/content/drive/My Drive/AML Project 2/Project Deliverable 3/imdb_with_glove_bert_embeddings.csv')
from google.colab import drive
drive.mount('/content/drive')
imdb_df = pd.read_csv('/content/drive/My Drive/AML Project 2/Project Deliverable 3/imdb_with_glove_bert_embeddings.csv')

# Read in orignal file, apply preprocessing to text
X = imdb_df['review']
# y = imdb_df['sentiment']
y = imdb_df['sentiment'].map({'positive': 1, 'negative': 0})
X_dev, X_test, y_dev, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_dev_tfidf, X_test_tfidf = get_tfidf_embedding(X_dev, X_test)
X_dev_bow, X_test_bow = get_bag_of_words_embedding(X_dev, X_test)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import torch
imdb_df['encode_glove'] = imdb_df['encode_glove'].apply(
    lambda x: np.fromstring(x.strip('[]'), sep=' ') if isinstance(x, str) else x
)
imdb_df['cls_bert'] = imdb_df['cls_bert'].apply(
    lambda x: np.fromstring(x.strip('tensor([])'), sep=',') if isinstance(x, str) else x.numpy() if isinstance(x, torch.Tensor) else x
)

# Vertically stack GloVe embeddings for train and test sets
X_dev_glove = np.vstack(imdb_df.loc[X_dev.index, 'encode_glove'].values)
X_test_glove = np.vstack(imdb_df.loc[X_test.index, 'encode_glove'].values)

# Vertically stack BERT embeddings for train and test sets
X_dev_bert = np.vstack(imdb_df.loc[X_dev.index, 'cls_bert'].values)
X_test_bert = np.vstack(imdb_df.loc[X_test.index, 'cls_bert'].values)

<ipython-input-12-6f2b5da5ddc8>:3: DeprecationWarning: string or file could not be read to its end due to unmatched data; this will raise a ValueError in the future.
  lambda x: np.fromstring(x.strip('[]'), sep=' ') if isinstance(x, str) else x


In [ ]:
imdb_df.head()

,review,sentiment,encode_glove,cls_bert
0,one reviewers mentioned watching 1 oz episode ...,positive,"[-0.22556999, 0.49417999, 0.48609999]","[-0.053433, 0.075857, 0.35144, 0.051112, -0.31..."
1,wonderful little production filming technique ...,positive,"[0.19742, 0.51139998, 0.19043]","[-0.24327, 0.10041, 0.16072, -0.13163, -0.2312..."
2,thought wonderful way spend time hot summer we...,positive,"[0.27976, 0.41418999, 0.30272999]","[-0.20687, 0.024594, 0.2932, 0.049245, -0.0782..."
3,basically family little boy jake thinks zombie...,negative,"[-0.089409, 0.18174, 0.59192002]","[-0.035247, 0.079194, 0.42486, -0.2258, -0.068..."
4,petter mattei love time money visually stunnin...,positive,"[-0.2457, -0.13846999, -0.30884999]","[-0.15308, 0.15589, 0.3954, 0.17397, 0.049523,..."


In [ ]:
pip install keras-tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 3.4 MB/s eta 0:00:00


In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Input
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import RandomizedSearchCV
import tensorflow as tf
from sklearn.metrics import classification_report
from keras_tuner import RandomSearch
from sklearn.metrics import roc_curve, auc

## TF-IDF

In [ ]:
# tfidf
model = Sequential()
model.add(Dense(units=64, activation='relu', input_dim=X_dev_tfidf.shape[1])) # try 128/64
model.add(Dense(units=32, activation='relu')) # try 64/32
model.add(Dropout(0.5)) #try 0.5/0.3
model.add(Dense(units=16, activation='relu')) # try 16/8
model.add(Dropout(0.5))
model.add(Dense(units=1, activation='sigmoid'))

NameError: name 'X_dev_tfidf' is not defined

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
with tf.device('/GPU:0'):  # Force the use of GPU
    history_tfidf = model.fit(
        X_dev_tfidf, y_dev,
        batch_size=32,
        epochs=10,
        validation_split=0.2,
        verbose=1)

Epoch 1/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 38s 37ms/step - accuracy: 0.7902 - loss: 0.4588 - val_accuracy: 0.8882 - val_loss: 0.2605
Epoch 2/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 34s 34ms/step - accuracy: 0.9668 - loss: 0.1181 - val_accuracy: 0.8792 - val_loss: 0.3655
Epoch 3/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 34s 34ms/step - accuracy: 0.9868 - loss: 0.0437 - val_accuracy: 0.8704 - val_loss: 0.5865
Epoch 4/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 34s 34ms/step - accuracy: 0.9929 - loss: 0.0241 - val_accuracy: 0.8696 - val_loss: 0.7105
Epoch 5/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 35s 35ms/step - accuracy: 0.9953 - loss: 0.0151 - val_accuracy: 0.8670 - val_loss: 0.9812
Epoch 6/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 34s 34ms/step - accuracy: 0.9956 - loss: 0.0113 - val_accuracy: 0.8624 - val_loss: 1.0956
Epoch 7/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 39s 39ms/step - accuracy: 0.9959 - loss: 0.0116 - val_accuracy: 0.8674 - val_loss: 1.2077
Epoch 8/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 36s 36ms/step - accuracy: 0.9972 -

In [ ]:
loss, accuracy = model.evaluate(X_test_tfidf, y_test, verbose=0)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

NameError: name 'model' is not defined

In [ ]:
y_pred = model.predict(X_test_tfidf)
y_pred_binary = (y_pred > 0.5).astype(int)
print(classification_report(y_test, y_pred_binary))

In [ ]:
fpr, tpr, thresholds = roc_curve(y_test, y_pred)
roc_auc = auc(fpr, tpr)

In [ ]:
# Hyperparameter Tuning - Tune manually above with the session crashed
def build_tfidf_model(hp):
    model = Sequential()
    model.add(Dense(units=hp.Choice('units_input', [64, 128]),
                    activation='relu', input_dim=X_dev_tfidf.shape[1]))
    model.add(Dense(units=hp.Choice('units_hidden', [32, 64]),
                    activation='relu'))
    model.add(Dropout(rate=hp.Choice('dropout', [0.2, 0.3])))
    model.add(Dense(units=1, activation='sigmoid'))
    model.compile(optimizer=tf.keras.optimizers.Adam(
                        hp.Choice('learning_rate', [1e-3, 1e-4])),
                  loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
tuner_tfidf = RandomSearch(
    build_tfidf_model,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=1,
    directory='my_dir',
    project_name='tfidf_quick_tune'
)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
with tf.device('/GPU:0'):
    tuner_tfidf.search(
        X_dev_tfidf.toarray(), y_dev,
        validation_split=0.2,
        epochs=5,  # Use only 5 epochs
        batch_size=32,
        verbose=1
    )


Search: Running Trial #1

Value             |Best Value So Far |Hyperparameter
64                |64                |units_input
32                |32                |units_hidden
0.3               |0.3               |dropout
0.0001            |0.0001            |learning_rate



In [ ]:
best_hps_tfidf = tuner_tfidf.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The optimal number of units in the input layer is {best_hps_tfidf.get('units_input')}.
The optimal learning rate is {best_hps_tfidf.get('learning_rate')}.
The optimal dropout rate is {best_hps_tfidf.get('dropout')}.
""")

# Build the best model
best_model_tfidf = tuner_tfidf.hypermodel.build(best_hps_tfidf)

# Train the best model
with tf.device('/GPU:0'):
    history_tfidf = best_model_tfidf.fit(
        X_dev_tfidf.toarray(), y_dev,  # Convert sparse matrix to dense
        validation_split=0.2,
        epochs=10,
        batch_size=32,
        verbose=1
    )

In [ ]:
y_pred = best_model_tfidf.predict(X_test_tfidf)
y_pred_binary = (y_pred > 0.5).astype(int)
print(classification_report(y_test, y_pred_binary))

## BOW

In [ ]:
# bow
model = Sequential()
model.add(Dense(units=64, activation='relu', input_dim=X_dev_bow.shape[1]))
model.add(Dense(units=32, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(units=16, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(units=1, activation='sigmoid'))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
history_bow = model.fit(X_dev_bow, y_dev,
                      batch_size=32,
                      epochs=10,
                      validation_split=0.2,
                      verbose=1)

In [ ]:
loss, accuracy = model.evaluate(X_test_bow, y_test, verbose=0)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

In [ ]:
y_pred = model.predict(X_test_bow)
y_pred_binary = (y_pred > 0.5).astype(int)
print(classification_report(y_test, y_pred_binary))

In [ ]:
fpr, tpr, thresholds = roc_curve(y_test, y_pred)
roc_auc = auc(fpr, tpr)

In [ ]:
# Hyperparameter Tuning - Tune manually above with the session crashed same as tf-idf
def build_bow_model(hp):
    model = Sequential()
    model.add(Dense(units=hp.Choice('units_input', [64, 128]),
                    activation='relu', input_dim=X_dev_tfidf.shape[1]))
    model.add(Dense(units=hp.Choice('units_hidden', [32, 64]),
                    activation='relu'))
    model.add(Dropout(rate=hp.Choice('dropout', [0.2, 0.3])))
    model.add(Dense(units=1, activation='sigmoid'))
    model.compile(optimizer=tf.keras.optimizers.Adam(
                        hp.Choice('learning_rate', [1e-3, 1e-4])),
                  loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
tuner_bow = RandomSearch(
    build_tfidf_model,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=1,
    directory='my_dir',
    project_name='tfidf_quick_tune'
)

In [ ]:
with tf.device('/GPU:0'):
    tuner_bow.search(
        X_dev_bow.toarray(), y_dev,
        validation_split=0.2,
        epochs=5,  # Use only 5 epochs
        batch_size=32,
        verbose=1
    )

In [ ]:
best_hps_bow = tuner_bow.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The optimal number of units in the input layer is {best_hps_bow.get('units_input')}.
The optimal learning rate is {best_hps_bow.get('learning_rate')}.
The optimal dropout rate is {best_hps_bow.get('dropout')}.
""")

# Build the best model
best_model_bow = tuner_bow.hypermodel.build(best_hps_tfidf)

# Train the best model
with tf.device('/GPU:0'):
    history_tfidf = best_model_bow.fit(
        X_dev_tfidf.toarray(), y_dev,  # Convert sparse matrix to dense
        validation_split=0.2,
        epochs=10,
        batch_size=32,
        verbose=1
    )

In [ ]:
y_pred = best_model_bow.predict(X_test_bow)
y_pred_binary = (y_pred > 0.5).astype(int)
print(classification_report(y_test, y_pred_binary))

## Glove Embedding

In [ ]:
# glove
model = Sequential()
model.add(Dense(units=64, activation='relu', input_dim=X_dev_glove.shape[1]))
model.add(Dense(units=32, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(units=16, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(units=1, activation='sigmoid'))

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
with tf.device('/GPU:0'):
  history_glove = model.fit(X_dev_glove, y_dev,
                      batch_size=32,
                      epochs=10,
                      validation_split=0.2,
                      verbose=1)

Epoch 1/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.5164 - loss: 0.6928 - val_accuracy: 0.5368 - val_loss: 0.6901
Epoch 2/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.5289 - loss: 0.6906 - val_accuracy: 0.5315 - val_loss: 0.6890
Epoch 3/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.5311 - loss: 0.6907 - val_accuracy: 0.5335 - val_loss: 0.6886
Epoch 4/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.5328 - loss: 0.6898 - val_accuracy: 0.5394 - val_loss: 0.6896
Epoch 5/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.5300 - loss: 0.6903 - val_accuracy: 0.5419 - val_loss: 0.6888
Epoch 6/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.5287 - loss: 0.6899 - val_accuracy: 0.5279 - val_loss: 0.6890
Epoch 7/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.5340 - loss: 0.6897 - val_accuracy: 0.5304 - val_loss: 0.6897
Epoch 8/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.5337 - loss: 0.6888 - 

In [ ]:
loss, accuracy = model.evaluate(X_test_glove, y_test, verbose=0)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

Test Loss: 0.688158392906189
Test Accuracy: 0.5385000109672546


In [ ]:
# Hyperparameter Tuning
def build_model(hp):
    model = Sequential()
    # Input layer with tunable units
    model.add(Dense(units=hp.Int('units_input', min_value=32, max_value=128, step=32),
                    activation='relu', input_dim=X_dev_glove.shape[1]))
    # Hidden layers
    for i in range(hp.Int('num_layers', 1, 3)):  # Tune the number of layers
        model.add(Dense(units=hp.Int(f'units_{i}', min_value=16, max_value=64, step=16),
                        activation=hp.Choice('activation', ['relu', 'tanh'])))
        model.add(Dropout(rate=hp.Float('dropout', min_value=0.1, max_value=0.5, step=0.1)))
    # Output layer
    model.add(Dense(units=1, activation='sigmoid'))
    # Compile with tunable learning rate
    model.compile(optimizer=tf.keras.optimizers.Adam(
                        hp.Choice('learning_rate', [1e-3, 1e-4, 1e-5])),
                  loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=20,  # Limit to 20 trials
    executions_per_trial=1,
    directory='my_dir',
    project_name='random_search'
)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
tuner.search(X_dev_glove, y_dev,
                 validation_split=0.2,
                 epochs=10,
                 batch_size=32,
                 verbose=1)

Trial 20 Complete [00h 00m 22s]
val_accuracy: 0.5413749814033508

Best val_accuracy So Far: 0.5463749766349792
Total elapsed time: 00h 07m 21s


In [ ]:
best_hps_glove = tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The optimal number of units in the input layer is {best_hps_glove.get('units_input')}.
The optimal learning rate is {best_hps_glove.get('learning_rate')}.
The optimal dropout rate is {best_hps_glove.get('dropout')}.
""")

# Build the best model using the best hyperparameters for GloVe
best_model_glove = tuner.hypermodel.build(best_hps_glove)

# Train the best model for GloVe
with tf.device('/GPU:0'):
    history_glove = best_model_glove.fit(
        X_dev_glove, y_dev,  # Use GloVe embeddings here
        validation_split=0.2,
        epochs=10,
        batch_size=32,
        verbose=1
    )


The optimal number of units in the input layer is 64.
The optimal learning rate is 0.001.
The optimal dropout rate is 0.30000000000000004.

Epoch 1/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.5125 - loss: 0.6923 - val_accuracy: 0.5290 - val_loss: 0.6899
Epoch 2/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.5241 - loss: 0.6918 - val_accuracy: 0.5404 - val_loss: 0.6897
Epoch 3/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.5244 - loss: 0.6912 - val_accuracy: 0.5421 - val_loss: 0.6890
Epoch 4/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.5349 - loss: 0.6895 - val_accuracy: 0.5421 - val_loss: 0.6881
Epoch 5/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.5407 - loss: 0.6896 - val_accuracy: 0.5425 - val_loss: 0.6883
Epoch 6/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.5326 - loss: 0.6897 - val_accuracy: 0.5401 - val_loss: 0.6886
Epoch 7/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.5372 - loss

In [ ]:
loss, accuracy = best_model_glove.evaluate(X_test_glove, y_test, verbose=0)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

Test Loss: 0.687380313873291
Test Accuracy: 0.5444999933242798


In [ ]:
y_pred = best_model_glove.predict(X_test_glove)
y_pred_binary = (y_pred > 0.5).astype(int)
print(classification_report(y_test, y_pred_binary))

313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
              precision    recall  f1-score   support

           0       0.55      0.42      0.48      4961
           1       0.54      0.66      0.60      5039

    accuracy                           0.54     10000
   macro avg       0.55      0.54      0.54     10000
weighted avg       0.55      0.54      0.54     10000



In [ ]:
fpr, tpr, thresholds = roc_curve(y_test, y_pred)
roc_auc = auc(fpr, tpr)

In [ ]:
roc_auc

0.5546663058980508

## Bert Embedding

In [ ]:
# bert
model = Sequential()
model.add(Dense(units=128, activation='relu', input_dim=X_dev_bert.shape[1]))
model.add(Dense(units=64, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(units=32, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(units=1, activation='sigmoid'))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
with tf.device('/GPU:0'):
  history_bert = model.fit(X_dev_bert, y_dev,
                      batch_size=32,
                      epochs=10,
                      validation_split=0.2,
                      verbose=1)

Epoch 1/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.7288 - loss: 0.5301 - val_accuracy: 0.8123 - val_loss: 0.4103
Epoch 2/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8070 - loss: 0.4256 - val_accuracy: 0.8173 - val_loss: 0.4050
Epoch 3/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8144 - loss: 0.4124 - val_accuracy: 0.8227 - val_loss: 0.3946
Epoch 4/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8116 - loss: 0.4139 - val_accuracy: 0.8185 - val_loss: 0.3962
Epoch 5/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8138 - loss: 0.4052 - val_accuracy: 0.8301 - val_loss: 0.3851
Epoch 6/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8291 - loss: 0.3901 - val_accuracy: 0.8290 - val_loss: 0.3897
Epoch 7/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8280 - loss: 0.3849 - val_accuracy: 0.8263 - val_loss: 0.3853
Epoch 8/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8324 - loss: 0.3806 - 

In [ ]:
loss, accuracy = model.evaluate(X_test_bert, y_test, verbose=0)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

Test Loss: 0.385319322347641
Test Accuracy: 0.828000009059906


In [ ]:
# Hyperparameter Tuning
def build_bert_model(hp):
    model = Sequential()
    # Input layer with tunable units
    model.add(Dense(units=hp.Int('units_input', min_value=64, max_value=256, step=32),
                    activation='relu', input_dim=X_dev_bert.shape[1]))
    # Hidden layers
    for i in range(hp.Int('num_layers', 1, 3)):  # Tune the number of layers
        model.add(Dense(units=hp.Int(f'units_{i}', min_value=32, max_value=128, step=32),
                        activation=hp.Choice('activation', ['relu', 'tanh'])))
        model.add(Dropout(rate=hp.Float('dropout', min_value=0.1, max_value=0.5, step=0.1)))
    # Output layer
    model.add(Dense(units=1, activation='sigmoid'))
    # Compile with tunable learning rate
    model.compile(optimizer=tf.keras.optimizers.Adam(
                        hp.Choice('learning_rate', [1e-3, 1e-4, 1e-5])),
                  loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
tuner = RandomSearch(
    build_bert_model,
    objective='val_accuracy',
    max_trials=20,  # Limit to 20 random trials
    executions_per_trial=1,  # Number of times to execute each trial
    directory='my_dir',
    project_name='bert_random_search'
)


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
with tf.device('/GPU:0'):
    tuner.search(
        X_dev_bert, y_dev,
        validation_split=0.2,
        epochs=10,
        batch_size=32,
        verbose=1)

Trial 20 Complete [00h 00m 25s]
val_accuracy: 0.8264999985694885

Best val_accuracy So Far: 0.8342499732971191
Total elapsed time: 00h 07m 44s


In [ ]:
best_hps_bert = tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The optimal number of units in the input layer is {best_hps_bert.get('units_input')}.
The optimal learning rate is {best_hps_bert.get('learning_rate')}.
The optimal dropout rate is {best_hps_bert.get('dropout')}.
""")

# Build the best model using the best hyperparameters
best_model_bert = tuner.hypermodel.build(best_hps_bert)

# Train the best model
with tf.device('/GPU:0'):
    history_bert = best_model_bert.fit(
        X_dev_bert, y_dev,
        validation_split=0.2,
        epochs=10,
        batch_size=32,
        verbose=1)


The optimal number of units in the input layer is 160.
The optimal learning rate is 0.0001.
The optimal dropout rate is 0.1.

Epoch 1/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.7085 - loss: 0.5550 - val_accuracy: 0.7976 - val_loss: 0.4324
Epoch 2/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8004 - loss: 0.4276 - val_accuracy: 0.8121 - val_loss: 0.4109
Epoch 3/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8127 - loss: 0.4088 - val_accuracy: 0.8184 - val_loss: 0.4000
Epoch 4/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8146 - loss: 0.4016 - val_accuracy: 0.8191 - val_loss: 0.3968
Epoch 5/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8222 - loss: 0.3922 - val_accuracy: 0.8242 - val_loss: 0.3913
Epoch 6/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8254 - loss: 0.3871 - val_accuracy: 0.8177 - val_loss: 0.3964
Epoch 7/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8277 - loss: 0.3828 - val

In [ ]:
loss, accuracy = best_model_bert.evaluate(X_test_bert, y_test, verbose=0)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

Test Loss: 0.3834768831729889
Test Accuracy: 0.8266000151634216


In [ ]:
y_pred = best_model_bert.predict(X_test_bert)
y_pred_binary = (y_pred > 0.5).astype(int)
print(classification_report(y_test, y_pred_binary))

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
              precision    recall  f1-score   support

           0       0.82      0.83      0.83      4961
           1       0.83      0.82      0.83      5039

    accuracy                           0.83     10000
   macro avg       0.83      0.83      0.83     10000
weighted avg       0.83      0.83      0.83     10000



In [ ]:
fpr, tpr, thresholds = roc_curve(y_test, y_pred)
roc_auc = auc(fpr, tpr)
roc_auc

0.9084736915393933

## Graph

In [ ]:
def plot_loss(histories, embedding_names):
    plt.figure(figsize=(10, 6))
    colors = ['blue', 'orange', 'green', 'purple']  # Colors for embeddings
    for idx, (history, name) in enumerate(zip(histories, embedding_names)):
        plt.plot(history.history['loss'], label=f'{name} - Training Loss', color=colors[idx], linestyle='-')
        plt.plot(history.history['val_loss'], label=f'{name} - Validation Loss', color=colors[idx], linestyle='--')
    plt.title('Training and Validation Loss for Different Embeddings')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))  # Move legend to the right
    plt.grid(True)
    plt.show()

In [ ]:
def plot_accuracy(histories, embedding_names):
    plt.figure(figsize=(10, 6))
    colors = ['blue', 'orange', 'green', 'purple']  # Colors for embeddings
    for idx, (history, name) in enumerate(zip(histories, embedding_names)):
        plt.plot(history.history['accuracy'], label=f'{name} - Training Accuracy', color=colors[idx], linestyle='-')
        plt.plot(history.history['val_accuracy'], label=f'{name} - Validation Accuracy', color=colors[idx], linestyle='--')
    plt.title('Training and Validation Accuracy for Different Embeddings')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))  # Move legend to the right
    plt.grid(True)
    plt.show()

In [ ]:
histories = [history_tfidf, history_bow, history_glove, history_bert]
embedding_names = ['TFIDF', 'Bag-of-Words', 'GloVe', 'BERT']

# Plot Loss
plot_loss(histories, embedding_names)

# Plot Accuracy
plot_accuracy(histories, embedding_names)